# Si symmetric tilt -exact

## import & prepare

In [1]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/Si_mp-149_conventional_standard.cif -o 'cif_files/Si_mp-149_conventional_standard.cif'

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/nmdl-mizo/interface_master.git (to revision develop) to /tmp/pip-req-build-_tof5zgf
  Running command git clone --filter=blob:none --quiet https://github.com/nmdl-mizo/interface_master.git /tmp/pip-req-build-_tof5zgf
  Resolved https://github.com/nmdl-mizo/interface_master.git to commit 2bdf96ed4835b036557ac18de82b4fa2afdabf24
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1059  100  1059    0     0  22531      0 --:--:-- --:--:-- --:--:-- 22531


In [14]:
from interfacemaster.cellcalc import get_primitive_hkl, rot
from interfacemaster.interface_generator import core, convert_vector_index
from numpy import array, dot, round, var, average, pi
from numpy.linalg import inv
from gb_code.csl_generator import print_list
from pymatgen.core.structure import Structure

## Define an interface core class

Input two Structure objects.For GBs, input the same structure; for interfaces, input two different structures.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL structure if you are not familiar with the indices in the primitive cell.

In [15]:
stct_1 = Structure.from_file('cif_files/Si_mp-149_conventional_standard.cif')
stct_2 = Structure.from_file('cif_files/Si_mp-149_conventional_standard.cif')

In [16]:
my_interface = core(stct_1, stct_2)

Warning!, this programme will rewrite the POSCAR file in this dir!


## Specify the rotation axis and searching parameters to find an approximate CSL

Since this is for a cubic lattice, we have determined rotation axis/angle to give a specific CSL

In [7]:
print_list([1,0,0],100)

Sigma:     1  Theta:   0.00 
Sigma:     5  Theta:  36.87 
Sigma:    13  Theta:  22.62 
Sigma:    17  Theta:  28.07 
Sigma:    25  Theta:  16.26 
Sigma:    29  Theta:  43.60 
Sigma:    37  Theta:  18.92 
Sigma:    41  Theta:  12.68 
Sigma:    53  Theta:  31.89 
Sigma:    61  Theta:  10.39 
Sigma:    65  Theta:  14.25 
Sigma:    73  Theta:  41.11 
Sigma:    85  Theta:   8.80 
Sigma:    89  Theta:  25.99 
Sigma:    97  Theta:  42.08 


In [8]:
#rotation matrix
R = rot([1,0,0],36.87/180*pi)

To calculate an exact CSL, we use the search_fixed function by setting exact=True

In [9]:
my_interface.parse_limit(du = 1e-2, S  = 1e-2, sgm1=100, sgm2=100, dd = 1e-2)
my_interface.search_fixed(R, exact=True, tol = 1e-3)

Congrates, we found an appx CSL!

U1 = 
[[ 1  0  2]
 [ 1  1 -1]
 [-1  1  0]]; sigma_1 = 5

U2 = 
[[ 1  1  2]
 [ 1  0 -1]
 [-1  1 -1]]; sigma_2 = 5

D = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]




'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, to form a twinning structure we need to deform the structure provided!

## Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [10]:
"""
The interface plane is the [0,-1,-2] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([0,-2,1]), my_interface.conv_lattice_1, my_interface.lattice_1, tol = 1e-3)
hkl

array([ 2, -1,  1])

## Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [11]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, \
                               lim = 50, tol_ortho = 1e-2, tol_integer = 1e-3)

hkl in CSL: [0 0 1]
cell 1:
[[-3 -1 -1]
 [ 3 -1  1]
 [-1  1  3]]
cell 2:
[[-3 -1  1]
 [ 3 -1 -1]
 [ 1  1  3]]


In [11]:
my_interface.bicrystal_structure.lattice

Lattice
    abc : 3.8669746532647453 3.8669746532647453 3.8669746532647453
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 40.88829284866483
      A : -2.734364 -2.734364 0.0
      B : -2.734364 0.0 -2.734364
      C : 1.6743150601518765e-16 -2.734364 -2.734364
    pbc : True True True

Here we obtained the two sets of indices to build a bicrystal

## Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [13]:
my_interface.get_bicrystal(xyz_1 = [5,1,1], xyz_2 = [5,1,1], filetype='VASP',dydz=  1 * my_interface.CNID[:,1])

In [13]:
help(my_interface.get_bicrystal)

Help on method get_bicrystal in module interfacemaster.interface_generator:

get_bicrystal(dydz=array([0., 0., 0.]), dx=0, dp1=0, dp2=0, xyz_1=[1, 1, 1], xyz_2=[1, 1, 1], vx=0, filename='POSCAR', two_D=False, filetype='VASP', mirror=False, KTI=False) method of interfacemaster.interface_generator.core instance
    generate a cif file for the bicrystal structure
    
    Parameters
    ----------
    dydz : numpy array
        translation vector in the interface
    dx : float
        translation normal to the interface
    dp1, dp2 : float
        termination of slab 1, 2
    xyz1, xyz2 : list
        expansion of slab 1, 2
    vx : float
        vacuum spacing, default 0
    filename : str
        filename, default 'POSCAR'
    two_D : bool
        whether a two CSL
    filetype : str
        filetype, 'VASP' or 'LAMMPS', default 'VASP'
    mirror : bool
        mirror, default False
    KTI : bool
        KTI, default False



In [13]:
my_interface.sample_lattice_planes(filetype='VASP')

terminating_sampling...
completed


Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [14]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
#To sample the CNID, try
my_interface.sample_CNID(grid = [4,4], xyz_1 = [1,1,1], xyz_2 = [1,1,1])

CNID
[[-1. -0.]
 [-1. -1.]
 [ 1. -1.]]
making 16 files...
completed


Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [ ]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

In [ ]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv